## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# get modules
from air_quality import *
from fisheries import *
from noise import *
from recreation import *
from scenic_resources import *
from soil_conservation import *
from transportation_sustainablecommunities import *
from vegetation_preservation import *
from water_quality import *
from wildlife import *

## Soil Conservation

## Water Quality

In [ ]:
# secchi depth data and chart
df = get_secchi_data_sql()
# df = get_secchi_data_web()
plot_secchi_depth(df, draft=False)

In [ ]:
# watercraft inspections
df = get_watercraft_inspection_data_web()
plot_watercraft_inspections(df, draft=False)

## Wildlife

In [ ]:
# goshawk data and chart
df = get_wildlife_data_web()
plot_goshawk_data(df, draft=False)

In [ ]:
# bald eagle winter roost data and chart
df = get_wildlife_data_web()
plot_bald_eagle_winter(df, draft=False)